# Training and Evaluation
This notebook presents a minimal example of how to setup, train and evaluate the deep learning architecture presented in: 
*"A Flexible Deep Learning Architecture for Temporal Sleep Stage Classification using Accelerometry and Photoplethysmography".* 

The notebook is organized following the brilliant work from Chambon et al. 2018 ^1.

Requirements: 
1. Install repository.
	- pip install -e
2. Install dependencies
	- dependencies
3. Prepare signal modalities and annotation in h5 files.
	- minimal_example/to_h5.py

###### ^2^ {{"S. Chambon, V. Thorey, P. J. Arnal, E. Mignot, A. Gramfort, "A deep learning architecture to detect events in EEG signals during sleep." IEEE 28th International Workshop on Machine Learning for Signal Processing (MLSP), 2018. [[Paper](https://arxiv.org/abs/1807.05981)|[Github](https://github.com/Dreem-Organization/dosed)]"| fndetail: 1}}



In [23]:
import os
import numpy as np

from datasets import get_train_validation_test, Dataset, BalancedDataset


seed = 2022

In [27]:
# data path is relative to the current working directory, which is found using: print(os.getcwd()) 

data_directory = '..\MasterAlgorithm\data\h5'
training_set, evaluation_set, test_set = get_train_validation_test(data_directory,
                                                    percent_test=40,
                                                    percent_validation=20,
                                                    seed=seed)

# Now we are ready 
print('training set: {}'.format(len(training_set)))
print('evaluation set: {}'.format(len(evaluation_set)))
print('test set: {}'.format(len(test_set)))

training set: 16
evaluation set: 4
test set: 12


# Dataset preparation
# ==============================================================================================================================================================

## Signal preprocessing. 
- "h5_path": string - path within h5 file to locate exact modality. 
- "channel_idx": list of integers - channel indexes to extract from the signal modality. 
- "preprocessing": list of dicts - defining preprocessing steps. A signal modality can have an arbitrary number of pre-processing steps.
- "batch_normalization": dict - assigning normalization actions to do during training. 
- "transformations": dict - assigning transformation (augmentation) operations to process the signal input with during training. 
- "add": boolean - assinging whether the different signal channels should be added after initial preprocessing (to reduce dimensionality).
- "fs_post": float - sample frequency of the preprocessed signal. 
- "dimensions": list of integers - dimensions of the preprocessed signal. 

In [20]:
signals_format = {
    'ACC_merge': {
        'h5_path': 'acc_signal',
        'channel_idx': [0, 1, 2],
        'preprocessing': [
            {
                'type': 'median',
                'args': {
                    'window_size': 30
                }
            },
            {
                'type': 'iqr_normalization_adaptive',
                'args': {
                    'median_window': 30001,
                    'iqr_window': 30001
                }
            },
            {
                'type': 'clip_by_iqr',
                'args': {
                    'threshold': 20
                }
            },
            {
                'type': 'cal_psd',
                'args': {
                    'window': 10 * 32, 
                    'noverlap': 8 * 32, 
                    'nfft': int(2 ** np.ceil(np.log2(10 * 32))),
                    'f_min': 0,
                    'f_max': 12,
                    'f_sub': 3
                }
            }

        ],
        'batch_normalization': {},
        'transformations': {
            #'image_translation': {},
            #'time_mask': {},
            #'freq_mask': {},
        },
        'add': True,
        'fs_post': 32,
        'dimensions': [int(2 ** np.ceil(np.log2(10 * 32)) / 32 * (6 - 0)) // 3, 1]
    },
    'ppg_signal': {
        'h5_path': 'ppg_signal',
        'channel_idx': [0],
        'preprocessing': [
            {
                'type': 'zscore',
                'args': {}
            },
            {
                'type': 'change_PPG_direction',
                'args': {}
            },
            {
                'type': 'iqr_normalization_adaptive',
                'args': {
                    'median_window': 301,
                    'iqr_window': 301
                }
            },
            {
                'type': 'clip_by_iqr',
                'args': {
                    'threshold': 20
                }
            },
            {
                'type': 'cal_psd',
                'args': {
                    'window': 10 * 32,  # 1/21, 1/35
                    'noverlap': 8 * 32,  # 1/42, 1/75
                    'nfft': int(2 ** np.ceil(np.log2(10 * 32))),
                    'f_min': 0.1,
                    'f_max': 2.1,
                    'f_sub': 1
                }
            }
        ],
        'batch_normalization': {},
        'transformations': {
            #'image_translation': {},
            #'time_mask': {},
            #'freq_mask': {},
        },
        'add': False,
        'fs_post': 1,
        'dimensions': [int(2 ** np.ceil(np.log2(10 * 32)) / 32 * (2.1 - 0.1)) // 1, 1]
    }
}

In [22]:

events = ['wake', 'light', 'deep', 'rem']
events_format = []
for event in events: 
    events_format += [{
        'name': event, 
        'h5_path': event,
        'probability': 1 / (len(events) + 1)
    }]

## Dataset class
The dataset class works as batch generator at training time. I handles preprocessing of the signal modalities, that are loaded from their specified h5 directories. 

- "records": list of strings - list of record filenames.
- "h5_directory": string - h5 directory of data files.
- "signal_format": list of dicts - directory and preprocessing steps for each signal modality (see above). 
- "window": interger - temporal window segment size (in seconds). 
- "number_of_channels": interger - number of signal modality inputs. 
- "events_format": list of dicts - format of events to model.  
- "prediction_resolution": integer - model output resolution (in seconds). 
- "overlap": float - consecutive window segments have this assigned overlap. Not used during balanced sampling. 
- "minimum_overlap": float - when signals are segmented, there is a risk of cutting off events. The minimum overlap is the required duration of an event that is cut off relative to the window size. 
- "batch_size": integer - batch size. 
- "mode": string - "inference" or "training".
- "cache_data": boolean - Cache preprocessing using Joblib.  
- "n_jobs": integer - starts parallel preprocessing. max is number of cores supported by local system.
- "seed": integer - seed.
- "use_mask": boolean - whether to apply mask. Mask must be defined as event in h5 file. 
- "load_signal_in_RAM": boolean - whether to load all preprocessed data in RAM during trianing (faster but requires memory).

In [28]:

dataset_params = {
    "h5_directory": data_directory, 
    "signals_format": signals_format,
    "window": 30 * 2 ** 10, 
    "number_of_channels": len(signals_format), 
    "events_format": events_format,
    "prediction_resolution": 30,
    "overlap": 0.25,
    "minimum_overlap": 0.1,
    "batch_size": 2,
    "mode": 'inference',
    "cache_data": True,
    "n_jobs": 4,
    "seed": seed,
    "use_mask": True,
    "load_signal_in_RAM": True
}

ds_train = Dataset(records=training_set, **dataset_params)
ds_evaluate = Dataset(records=evaluation_set, **dataset_params)
ds_test = Dataset(records=test_set, **dataset_params)

  0%|          | 0/16 [00:00<?, ?it/s]

 75%|███████▌  | 12/16 [00:53<00:17,  4.47s/it]

100%|██████████| 16/16 [01:47<00:00,  7.39s/it]

100%|██████████| 16/16 [01:47<00:00,  6.73s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 668.55it/s]

TODO - Display input/output from each preprocessing method!